In [1]:
import numpy as np
from astropy.io import fits
from scipy.interpolate import interp1d
import math as m
import scipy.constants as p
import pandas as pd
import matplotlib.pyplot as plt
import timeit
import sys
sys.path.append("..")
import physics
sys.path.append("../..")
from spec.spectrum import *
#%run 'Electrons_v1.ipynb' might need things form this later
import xsecs
#something for precision

In [2]:
class Rates(object):
    '''
    Allows for easier storage of all xsecs/rates (probabilities)
    Made specifically for ion and heating ONLY.
    
    Attributes
    -----------
    E_spec : Spectrum class
        Electron energy and dNdE spectrum
    n : int
        The bin of interest in the spectrum
    den : ndarray
        density of [H, He, He+] in cm^-3
    rates : Filled with various rates in s^-1
    '''
    
    def __init__(self, E_spec, n, den=[1,1,1], rates=np.zeros(4)):
        '''Return a Rate object with electron of energy *E_spec.eng[n]*
        in a universe with densities *den*; initializes *rate* vector'''
        self.energy = E_spec.eng[n]
        self.n = n
        self.dNdE = E_spec.dNdE[n]
        self.den = den
        self.rates = rates
        self.v = np.sqrt(2*np.array([E_spec.eng[n]])/physics.me) 
        self.mult = self.den*self.v
    
    def ion_rate_calc(self):
        '''Fills *self.rate[1:3]* vector by calculating xsec and then converting to rate
        '''
        ion_rates = xsecs.ionize_cs(self.energy*np.ones(3),np.array([1,2,3]))*self.mult
        self.rates = np.append([0], ion_rates)
        return self.rates
    
    def heat_rate_calc(self, x_e, rs):
        '''Fills *self.rate[0]* vector with fraction going to heating
            x_e and rs...
        '''
        dE = xsecs.heating_dE(self.energy, x_e, rs, nH=physics.nH)
        delta_dNdE = np.zeros(len(E_spec.dNdE))
        np.put(delta_dNdE, self.n, self.dNdE)
        delta = Spectrum(E_spec.eng, delta_dNdE, rs)
        shift_delta_eng = E_spec.eng+dE
        delta.shift_eng(shift_delta_eng)
        delta.rebin(E_spec.eng)
        
        heating_frac = delta.dNdE[self.n]/self.dNdE
        self.rates[0] = 1-heating_frac
        return(self.rates)
    def tot_sum(self):
        '''Sums all elements of rate
        '''
        tot_sum = sum(self.rates)
        return tot_sum
    def con_prob(self):
        '''Finds cumulative probability of each process
        '''
        con_prob = self.rates/self.tot_sum()
        return con_prob

In [3]:
eng1 = np.logspace(0.,4.,num = 500)
dNdE1 = np.logspace(0.,5.,num = 500)
rs=1000
E_spec = Spectrum(eng1,dNdE1,rs)
photon_spec = Spectrum(E_spec.eng,np.zeros(len(E_spec.eng)),rs)
TM=np.zeros([len(photon_spec.eng)+2,len(E_spec.eng)]) #possibly make pandas
Transfer_Matrix=pd.DataFrame(TM) #or np.zeros
#print(Transfer_Matrix)
print(np.array([E_spec.eng[330]]))
print(xsecs.ionize_cs(np.array([E_spec.eng[100]]),np.ones(1)))

[ 441.86599564]
[ 0.]


In [4]:
a = Rates(E_spec, 430)
print(a.energy)
print(a.ion_rate_calc())
print(a.heat_rate_calc(0.0001, 300))
print(a.tot_sum())
print(a.con_prob())

2798.29687566
[  0.00000000e+00   5.36837606e-19   6.25760697e-19   1.09194109e-19]
[  1.44947387e-11   5.36837606e-19   6.25760697e-19   1.09194109e-19]
1.44947400144e-11
[  9.99999912e-01   3.70367186e-08   4.31715709e-08   7.53336102e-09]


../spec/spectrum.py:381: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if bound_arr == None:
../spec/spectrum.py:469: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if bound_arr == None:
